In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pyspeckit

from matplotlib.colors import ListedColormap

## Read data files

In [ ]:
nh3_11, hd_amm = fits.getdata('datacubes/nh3_11_whole_cube_8as_3px.fits', header=True)
npeaks, hd2d = fits.getdata('bayesian_fit_files/npeaks_cut5.fits', header=True)
snr = fits.getdata('bayesian_fit_files/snr11.fits')

#### Area masks

In [ ]:
from os import listdir

fls = listdir('data_files_prev_works/masks_anika')
fls = [i for i in fls if 'rgrd.fits' in i]

msk_regs = {ms[:-10] : fits.getdata('data_files_prev_works/masks_anika/'+ms) for ms in fls}

### Aplpy plots

In [ ]:
from aplpy import FITSFigure
from astropy.io.fits import PrimaryHDU
import astropy.units as u

import matplotlib as mpl
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

In [ ]:
def aplpy_plot(fig= None, data=None, hd=None, cmap='inferno', vmin=9.7, vmax=10.7, colourbar=True, bm_clr='k',
               colourbar_text=r'$\rm{v_{LSR}}\ (km\,s^{-1})$', beam_colour='green', fgsz=(7,7), zoom=False, 
               colourbar_loc='right', scalebar_corner='top right', sbplt=[0, 0, 1, 1], contours=False, 
               contour_clr='k', background_grey=False, bckgr_gr_clr='0.8', x_bm_lbl=0.07, y_bm_lbl=0.07, 
               colourscale_stretch='linear', vmid=None) :
    """ 
    return aplpy FITSfigure for given data
    """
    if not fig:
        fig = plt.figure(figsize=fgsz)
        
    fig = FITSFigure( data=PrimaryHDU(data=data, header=hd), figure=fig, subplot=sbplt)
    
    if background_grey:
        cmap = plt.get_cmap(cmap)
        cmap.set_bad(bckgr_gr_clr)
    
    fig.show_colorscale( cmap=cmap, vmin=vmin, vmax=vmax,stretch=colourscale_stretch, vmid=vmid)
    
    if colourbar:
        fig.add_colorbar()
        fig.colorbar.show(location=colourbar_loc, axis_label_text=colourbar_text)
    
    fig.add_beam(color=beam_colour)
    
    distance = 302 *u.pc
    scale=(0.1*u.pc/distance*u.rad).to(u.deg)
    fig.add_scalebar(scale, label='0.1 pc', corner=scalebar_corner)
#     fig.set_nan_color('0.7')
    fig.ticks.set_color('black')
    fig.add_label( x_bm_lbl, y_bm_lbl, 'B5', relative=True, color= bm_clr, size=15)
    
    if zoom:
        fig.recenter(56.914, 32.879, width=0.13, height=0.138)
        
    if contours :
        fig.show_contour(msk_regs['fil1'], colors=[contour_clr], levels=[0])
        fig.show_contour(msk_regs['fil2'], colors=[contour_clr], levels=[0])

        fig.show_contour(msk_regs['cnd1'], colors=[contour_clr], levels=[0], linestyles='dashed')
        fig.show_contour(msk_regs['cnd2'], colors=[contour_clr], levels=[0], linestyles='dashed')
        fig.show_contour(msk_regs['cnd3'], colors=[contour_clr], levels=[0], linestyles='dashed')
    
    return fig

### calculate simple moment 0

In [ ]:
sum_11 = np.nansum(nh3_11, axis=0)

dv_ms = (hd_amm['CDELT3']/hd_amm['CRVAL3']) * 299792458
dv_kms = dv_ms/1000

integ_inten = sum_11 * dv_kms.value

integ_inten[integ_inten == 0] = np.nan

In [ ]:
fig = aplpy_plot(data=integ_inten, hd=hd2d, cmap='inferno', zoom=True, fgsz=(6,10), vmin=0, vmax=15, contours=True, 
                 contour_clr='k', background_grey=True, colourbar_text=r'Integrated Intensity (K km$\,s^{-1}$)', 
                 colourscale_stretch='log', vmid=-2, bckgr_gr_clr='0.5')

fig.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='w')


fig.show_arrows(56.92, 32.92, 0.02, 0.002, width=0.3, head_width=2, head_length=2, color='w')
fig.add_label( 0.18, 0.82, 'clump-1', relative=True, color='w',size=13, weight=1000, horizontalalignment='left')

fig.add_label( 0.44, 0.65, 'Fil-1', relative=True, color='w',size=13, weight=1000, rotation=70, 
              horizontalalignment='left')
fig.add_label( 0.52, 0.28, 'Fil-2', relative=True, color='w',size=13, weight=1000, horizontalalignment='left')


fig.show_arrows(56.915, 32.873, -0.02, 0.006, width=0.3, head_width=2, head_length=2, color='w')
fig.add_label( 0.63, 0.5, 'cond-1', relative=True, color='w',size=13, weight=1000, horizontalalignment='left')
fig.show_arrows(56.925, 32.868, 0.02, 0.006, width=0.3, head_width=2, head_length=2, color='w')
fig.add_label( 0.26, 0.479, 'cond-2', relative=True, color='w',size=13, weight=1000)
fig.show_arrows(56.927, 32.858, 0.02, -0.003, width=0.3, head_width=2, head_length=2, color='w')
fig.add_label( 0.25, 0.3, 'cond-3', relative=True, color='w',size=13, weight=1000)

# fig.show_arrows(56.92, 32.92, 0.025, 0.002, width=0.3, head_width=2, head_length=2, color='w')


fig.savefig('plots/mom0_sel-chan_w-labels.pdf', dpi=1000)

### Show example spectra with 2, 3 component fits

In [ ]:
from astropy.wcs import WCS
wcs = WCS(hd2d)

from matplotlib.patches import Circle

In [ ]:
xlist = np.array([115, 125, 94])
ylist = np.array([73, 59, 94])
labels = np.array([1, 2, 3])

dxlist = np.array([30, 15, -40])
dylist = np.array([10, -40, 20])
ra_list, dec_list = wcs.all_pix2world(xlist, ylist, 0)
ra_list_text, dec_list_text = wcs.all_pix2world(xlist + dxlist, ylist + dylist, 0)
dra_list = ra_list - ra_list_text
ddec_list = dec_list - dec_list_text

In [ ]:
cmaplst = ['#f0f9e8', '#ccebc5', '#7bccc4', '#2b8cbe'] #['#f0f9e8', '#7bccc4', '#43a2ca', '#0868ac'] #['#f1f7d2', '#7fcdbb', '#2c7fb8']  # stolen from colorbrewer2
lcmap = ListedColormap(cmaplst)
lcmap.set_bad('#f0f0f0')

npeaks_snr5 = npeaks.copy()
npeaks_snr5[npeaks_snr5 == 0] = 10
npeaks_snr5 = npeaks_snr5*(snr>4)
npeaks_snr5[npeaks_snr5 == 0] = np.nan
npeaks_snr5[npeaks_snr5 == 10] = 0

fig = aplpy_plot(data=npeaks_snr5, hd=hd2d, cmap=lcmap, zoom=True, fgsz=(6,10), vmin=-0.5, vmax=3.5, 
                 colourbar_text='number of components fit', background_grey=True, beam_colour='k', contours=True)

fig.colorbar.set_ticks([0, 1, 2, 3])
# fig.colorbar.set_ticklabel(['e', 'e', 'e', 'e'])


fig.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')

for i in range(len(labels)):
    fig.add_label( ra_list_text[i], dec_list_text[i], labels[i], color='k', size=14, weight=500, 
                  bbox=dict(boxstyle="circle", fc="0.8", alpha=1))
    
    fig.show_arrows(ra_list_text, dec_list_text, dra_list, ddec_list, head_width=2, width=0.1, color='k')
#     fig.show_arrows(ra_list, dec_list, dra_list*(-1), ddec_list*(-1), head_width=1, width=0.1, color='k'
#                    )
#                     arrowstyle='CurveA')
#                     arrowprops=dict(arrowstyle="-", shrinkA=0, shrinkB=0, connectionstyle="arc3, rad=0.2", alpha=0.8))
    
# fig.savefig('plots/npeaks_w-contours_w-ex-spec-loc.pdf', dpi=1000)

In [ ]:
arr1_1 = fits.getdata('sorted_comps/first_sort_vel-sig-tmb_extended.fits')

dark_blue_arr = fits.getdata('sorted_comps/dark_blue_vel-sig-tmb.fits')
mid_blue_arr = fits.getdata('sorted_comps/mid_blue_vel-sig-tmb.fits')
mid_red_arr = fits.getdata('sorted_comps/mid_red_vel-sig-tmb.fits')
dark_red_arr = fits.getdata('sorted_comps/dark_red_vel-sig-tmb.fits')

In [ ]:
vel_ext = arr1_1[0]*(snr>4)
vel_ext[vel_ext == 0] = np.nan

fig = aplpy_plot(data=vel_ext, hd=hd2d, cmap='RdYlBu_r', zoom=True, fgsz=(6,10), contours=True, background_grey=True)

# fig.add_label( 0.534, 0.52, 'cond-1', relative=True, color='k',size=10, weight=500, rotation=70, horizontalalignment='left')
# fig.add_label( 0.38, 0.47, 'cond-2', relative=True, color='k',size=10, weight=500, rotation=60)
# fig.add_label( 0.45, 0.28, 'cond-3', relative=True, color='k',size=10, weight=500, rotation=310)

fig.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')


fig.savefig('plots/vel_extended_w-coh-cont.pdf', dpi=1000)

In [ ]:
sig_ext = arr1_1[1]*(snr>4)
sig_ext[sig_ext == 0] = np.nan

fig = aplpy_plot(data=sig_ext, hd=hd2d, cmap='inferno', zoom=True, fgsz=(6,10), vmin=0, vmax=0.5, contours=True, 
                 contour_clr='w', background_grey=True, colourbar_text=r'$\sigma_v\ (km\,s^{-1}$)')

# fig.add_label( 0.534, 0.52, 'cond-1', relative=True, color='w',size=10, weight=1000, rotation=70, 
#               horizontalalignment='left')
# fig.add_label( 0.38, 0.47, 'cond-2', relative=True, color='w',size=10, weight=1000, rotation=60)
# fig.add_label( 0.45, 0.28, 'cond-3', relative=True, color='w',size=10, weight=1000, rotation=310)

fig.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='w', linestyles='dotted', linewidths=3)
fig.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted', linewidths=1)
# fig.show_contour('../../coherent_core_bound_SD.fits', levels=[0.5], colors='g', linestyles='dotted')


fig.savefig('plots/sig_extended_w-coh-cont.pdf', dpi=200)

In [ ]:
fig = plt.figure(figsize=(6,9))

fig1 = aplpy_plot(fig=fig, data=dark_blue_arr[0], hd=hd2d, cmap='RdYlBu_r', zoom=True, sbplt=[0.12, 0.5, 0.4, 0.4], 
                  colourbar=False, vmin=9.7, vmax=10.7, contours=True, background_grey=True, y_bm_lbl=0.1)

fig1.tick_labels.hide_x()
fig1.axis_labels.hide()


fig2 = aplpy_plot(fig=fig, data=mid_blue_arr[0], hd=hd2d, cmap='RdYlBu_r', zoom=True, sbplt=[0.55, 0.5, 0.4, 0.4], 
                  colourbar=False, vmin=9.7, vmax=10.7, contours=True, background_grey=True, y_bm_lbl=0.1)
fig2.tick_labels.hide()
fig2.axis_labels.hide()

fig3 = aplpy_plot(fig=fig, data=mid_red_arr[0], hd=hd2d, cmap='RdYlBu_r', zoom=True, sbplt=[0.12, 0.07, 0.4, 0.4], 
                  colourbar=False, vmin=9.7, vmax=10.7, contours=True, background_grey=True, y_bm_lbl=0.1)
fig3.axis_labels.hide_y()

fig4 = aplpy_plot(fig=fig, data=dark_red_arr[0], hd=hd2d, cmap='RdYlBu_r', zoom=True, sbplt=[0.55, 0.07, 0.4, 0.4], 
                  colourbar=False, vmin=9.7, vmax=10.7, contours=True, background_grey=True, y_bm_lbl=0.1)
fig4.tick_labels.hide_y()
fig4.axis_labels.hide_y()

fig1.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig2.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig3.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig4.recenter(56.907, 32.8833, width=0.07, height=0.11)


fig1.add_label( 0.8, 0.85, 'far-blue', relative=True, color='b', size=15, weight=500)
fig2.add_label( 0.8, 0.85, 'mid-blue', relative=True, color='dodgerblue', size=15, weight=500)
fig3.add_label( 0.8, 0.85, 'mid-red', relative=True, color='#ef6548', size=15, weight=500)
fig4.add_label( 0.8, 0.85, 'far-red', relative=True, color='r', size=15, weight=500)


fig1.add_label( 0.12, 0.93, '(a)', relative=True, color='k', size=14, weight=500)
fig2.add_label( 0.12, 0.93, '(b)', relative=True, color='k', size=14, weight=500)
fig3.add_label( 0.12, 0.93, '(c)', relative=True, color='k', size=14, weight=500)
fig4.add_label( 0.12, 0.93, '(d)', relative=True, color='k', size=14, weight=500)

fig1.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig2.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig3.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig4.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')

plt.tight_layout()
fig.savefig('plots/vel_4-extra-comp_w-coh-cont_new.pdf', dpi=1000)

In [ ]:
fig = plt.figure(figsize=(6,10))

fig1 = aplpy_plot(fig=fig, data=dark_blue_arr[1], hd=hd2d, cmap='inferno', zoom=True, sbplt=[0.12, 0.52, 0.4, 0.425], 
                  colourbar=False, vmin=0, vmax=0.4, contours=True, contour_clr='w', background_grey=True, y_bm_lbl=0.1)

fig1.tick_labels.hide_x()
fig1.axis_labels.hide_x()


fig2 = aplpy_plot(fig=fig, data=mid_blue_arr[1], hd=hd2d, cmap='inferno', zoom=True, sbplt=[0.55, 0.52, 0.4, 0.425], 
                  colourbar=False, vmin=0, vmax=0.4, contours=True, contour_clr='w', background_grey=True, y_bm_lbl=0.1)
fig2.tick_labels.hide()
fig2.axis_labels.hide()

fig3 = aplpy_plot(fig=fig, data=mid_red_arr[1], hd=hd2d, cmap='inferno', zoom=True, sbplt=[0.12, 0.07, 0.4, 0.425], 
                  colourbar=False, vmin=0, vmax=0.4, contours=True, contour_clr='w', background_grey=True, y_bm_lbl=0.1)

fig4 = aplpy_plot(fig=fig, data=dark_red_arr[1], hd=hd2d, cmap='inferno', zoom=True, sbplt=[0.55, 0.07, 0.4, 0.425], 
                  colourbar=False, vmin=0, vmax=0.4, contours=True, contour_clr='w', background_grey=True, y_bm_lbl=0.1)
fig4.tick_labels.hide_y()
fig4.axis_labels.hide_y()

fig1.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig2.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig3.recenter(56.907, 32.8833, width=0.07, height=0.11)
fig4.recenter(56.907, 32.8833, width=0.07, height=0.11)

fig1.add_label( 0.8, 0.5, 'far-blue', relative=True, color='b', size=15, weight=500)
fig2.add_label( 0.8, 0.5, 'mid-blue', relative=True, color='dodgerblue', size=15, weight=500)
fig3.add_label( 0.8, 0.5, 'mid-red', relative=True, color='#ef6548', size=15, weight=500)
fig4.add_label( 0.8, 0.5, 'far-red', relative=True, color='r', size=15, weight=500)


fig1.add_label( 0.12, 0.93, '(a)', relative=True, color='k', size=14, weight=500)
fig2.add_label( 0.12, 0.93, '(b)', relative=True, color='k', size=14, weight=500)
fig3.add_label( 0.12, 0.93, '(c)', relative=True, color='k', size=14, weight=500)
fig4.add_label( 0.12, 0.93, '(d)', relative=True, color='k', size=14, weight=500)

fig1.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig2.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig3.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')
fig4.show_contour('data_files_prev_works/coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')

plt.tight_layout()
fig.savefig('plots/sig_4-extra-comp_w-coh-cont.pdf', dpi=1000)

### Bayes K maps

In [ ]:
K_arr_12 = fits.getdata('Tkmin6K/sig_u-lim_ext/B5-Ks.fits')
K_arr_34 = fits.getdata('Tkmin6K/B5-Ks.fits')

In [ ]:
K_arr_01_snr5 = K_arr_12[0].copy()
K_arr_01_snr5[K_arr_01_snr5 == 0] = -2
K_arr_01_snr5 = K_arr_01_snr5*(snr>4)
K_arr_01_snr5[K_arr_01_snr5 == 0] = np.nan
K_arr_01_snr5[K_arr_01_snr5 == -2] = 0

In [ ]:
K_arr_12_snr5 = K_arr_12[1].copy()
K_arr_12_snr5[K_arr_12_snr5 == 0] = -2
K_arr_12_snr5 = K_arr_12_snr5*(snr>4)
K_arr_12_snr5[K_arr_12_snr5 == 0] = np.nan
K_arr_12_snr5[K_arr_12_snr5 == -2] = 0

In [ ]:
K_arr_23_snr5 = K_arr_34[2].copy()
K_arr_23_snr5[K_arr_23_snr5 == 0] = -2
K_arr_23_snr5 = K_arr_23_snr5*(snr>4)
K_arr_23_snr5[K_arr_23_snr5 == 0] = np.nan
K_arr_23_snr5[K_arr_23_snr5 == -2] = 0

In [ ]:
lcmap = 'viridis'


fig = aplpy_plot(data=K_arr_01_snr5, hd=hd2d, cmap=lcmap, zoom=True, fgsz=(6,10), vmin=-2, vmax=2000, vmid=-3,
                 colourbar_text=r'ln $K^1_0$', background_grey=True, beam_colour='k', contours=True, 
                colourscale_stretch='log')

fig.colorbar.set_ticks([0, 10, 100, 1000])
# fig.colorbar.set_ticklabel(['e', 'e', 'e', 'e'])


# fig.show_contour('../../coherent_core_bound_SD.fits', levels=[0.5], colors='k', linestyles='dotted')

    
fig.savefig('plots/K01_arr_w-contours.pdf', dpi=200)

In [ ]:
lcmap = 'viridis'


fig = aplpy_plot(data=K_arr_12_snr5, hd=hd2d, cmap=lcmap, zoom=True, fgsz=(6,10), vmin=0, vmax=100, vmid=-1,
                 colourbar_text=r'ln $K^2_1$', background_grey=True, beam_colour='k', contours=False, 
                colourscale_stretch='log')

fig.colorbar.set_ticks([0, 5, 10, 100])
# fig.colorbar.set_ticklabel(['e', 'e', 'e', 'e'])


fig.show_contour(msk_regs['fil1'], colors=['w'], levels=[0])
fig.show_contour(msk_regs['fil2'], colors=['w'], levels=[0])

fig.show_contour(msk_regs['cnd1'], colors=['k'], levels=[0], linestyles='dashed')
fig.show_contour(msk_regs['cnd2'], colors=['k'], levels=[0], linestyles='dashed')
fig.show_contour(msk_regs['cnd3'], colors=['k'], levels=[0], linestyles='dashed')
    
fig.savefig('sig_u-lim_ext/plots/K12_arr_w-contours.pdf', dpi=200)

In [ ]:
lcmap = 'viridis'


fig = aplpy_plot(data=K_arr_23_snr5, hd=hd2d, cmap=lcmap, zoom=True, fgsz=(6,10), vmin=-1, vmax=20, vmid=-3,
                 colourbar_text=r'ln $K^3_2$', background_grey=True, beam_colour='k', contours=True, 
                colourscale_stretch='log', contour_clr='w')

fig.colorbar.set_ticks([0, 5, 10])
# fig.colorbar.set_ticklabel(['e', 'e', 'e', 'e'])


fig.show_contour(msk_regs['fil1'], colors=['w'], levels=[0])
fig.show_contour(msk_regs['fil2'], colors=['w'], levels=[0])

fig.show_contour(msk_regs['cnd1'], colors=['k'], levels=[0], linestyles='dashed')
fig.show_contour(msk_regs['cnd2'], colors=['w'], levels=[0], linestyles='dashed')
fig.show_contour(msk_regs['cnd3'], colors=['k'], levels=[0], linestyles='dashed')
    
fig.savefig('sig_u-lim_ext/plots/K23_arr_w-contours.pdf', dpi=200)